In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%load_ext autoreload
%autoreload 2

## Check clip sizes

In [4]:
from Cfg import Cfg
C = Cfg('NIST', 16000, 'pashto', 'build') 

splits=C.split_files()

L=[x['t_seconds'] for x in splits]

In [3]:
min(L),max(L),sum(L)/len(L)

(0.35, 31.19, 4.100249459529579)

## Train step 0: Bootstrap from pretrained model (one time)

In [16]:
from load_pretrained_amharic_model import load_pretrained_amharic_model
C1 = Cfg('NIST', 16000, 'amharic', 'build') 
model = load_pretrained_amharic_model(C1, 0)

[NeMo W 2020-10-17 01:11:55 modelPT:102] Please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: NIST/openasr20_amharic/build/train_manifest.json
    sample_rate: 16000
    labels:
    - ' '
    - ሀ
    - ሁ
    - ሂ
    - ሃ
    - ሄ
    - ህ
    - ሆ
    - ለ
    - ሉ
    - ሊ
    - ላ
    - ሌ
    - ል
    - ሎ
    - ሏ
    - ሐ
    - ሑ
    - ሒ
    - ሔ
    - ሕ
    - መ
    - ሙ
    - ሚ
    - ማ
    - ሜ
    - ም
    - ሞ
    - ሟ
    - ሠ
    - ሡ
    - ሣ
    - ሥ
    - ሦ
    - ረ
    - ሩ
    - ሪ
    - ራ
    - ሬ
    - ር
    - ሮ
    - ሯ
    - ሰ
    - ሱ
    - ሲ
    - ሳ
    - ሴ
    - ስ
    - ሶ
    - ሷ
    - ሸ
    - ሹ
    - ሺ
    - ሻ
    - ሼ
    - ሽ
    - ሾ
    - ሿ
    - ቀ
    - ቁ
    - ቂ
    - ቃ
    - ቄ
    - ቅ
    - ቆ
    - ቋ
    - በ
    - ቡ
    - ቢ
    - ባ
    - ቤ
    - ብ
    - ቦ
    - ቧ
    - ቨ
    - ቪ
    - ቫ
    - ቬ
    - ቭ
    - ተ
    - ቱ
    - ቲ
    - ታ
    - ቴ
    - ት
    - ቶ
    - ቷ
    - ቸ
  

[NeMo I 2020-10-17 01:11:55 features:241] PADDING: 16
[NeMo I 2020-10-17 01:11:55 features:258] STFT using torch
loaded save/nemo_amharic/amharic_20201017_001951_101244_epoch=05.ckpt


In [17]:
model

EncDecCTCModel(
  (preprocessor): AudioToMelSpectrogramPreprocessor(
    (featurizer): FilterbankFeatures()
  )
  (encoder): ConvASREncoder(
    (encoder): Sequential(
      (0): JasperBlock(
        (mconv): ModuleList(
          (0): MaskedConv1d(
            (conv): Conv1d(64, 64, kernel_size=[33], stride=[2], padding=(16,), dilation=[1], groups=64, bias=False)
          )
          (1): MaskedConv1d(
            (conv): Conv1d(64, 256, kernel_size=(1,), stride=(1,), bias=False)
          )
          (2): BatchNorm1d(256, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        )
        (mout): Sequential(
          (0): ReLU()
          (1): Dropout(p=0.0, inplace=False)
        )
      )
      (1): JasperBlock(
        (mconv): ModuleList(
          (0): MaskedConv1d(
            (conv): Conv1d(256, 256, kernel_size=[33], stride=[1], padding=(16,), dilation=[1], groups=256, bias=False)
          )
          (1): MaskedConv1d(
            (conv): Conv1d(256, 256, ke

In [18]:
from ruamel.yaml import YAML
from omegaconf import DictConfig
config_path = f'{C.language}_{C.sample_rate}.yaml'
yaml = YAML(typ='safe')
with open(config_path) as f:
    params = yaml.load(f)
train_manifest=f'{C.build_dir}/train_manifest.json'
test_manifest=f'{C.build_dir}/test_manifest.json'
params['model']['train_ds']['manifest_filepath'] = train_manifest
params['model']['validation_ds']['manifest_filepath'] = test_manifest
model.change_vocabulary(new_vocabulary=params['labels'])

[NeMo I 2020-10-17 01:12:00 ctc_models:189] Changed decoder to output to ['غ', 'ش', 'ک', 'ژ', 'ړ', 'ج', 'چ', 'ښ', 'ض', 'ل', 'ئ', 'ڼ', 'ء', 'ې', 'ع', 'ذ', 'ی', 'آ', 'ث', 'ا', 'م', 'څ', 'ه', 'د', 'خ', 'پ', 'ټ', 'س', 'ص', 'ق', 'ز', 'ً', 'ف', 'ت', 'ۍ', 'ګ', 'ځ', 'ي', 'ب', 'ن', 'ر', 'ط', ' ', 'ډ', 'و', 'ظ', 'ح', 'ږ'] vocabulary.


In [20]:
import pytorch_lightning as pl
import os, datetime

class ModelCheckpointAtEpochEnd(pl.callbacks.ModelCheckpoint):
    def on_epoch_end(self, trainer, pl_module):
        metrics = trainer.callback_metrics
        metrics['epoch'] = trainer.current_epoch
        trainer.checkpoint_callback.on_validation_end(trainer, pl_module)

pid=os.getpid()
dt=datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

checkpoint_callback = ModelCheckpointAtEpochEnd(
    filepath=C.model_save_dir+'/amharic_'+f'{dt}_{pid}'+'_{epoch:02d}',
    verbose=True,
    save_top_k=-1,
    save_weights_only=False,
    period=1)

trainer = pl.Trainer(gpus=[0], max_epochs=1000, amp_level='O1', precision=16, checkpoint_callback=checkpoint_callback)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]
Using native 16bit precision.


In [22]:
model.set_trainer(trainer)
model.setup_training_data(train_data_config=params['model']['train_ds'])
model.setup_validation_data(val_data_config=params['model']['validation_ds'])
model.setup_optimization(optim_config=DictConfig(params['model']['optim']))

[NeMo I 2020-10-17 01:18:44 collections:173] Dataset loaded with 7805 files totalling 8.88 hours
[NeMo I 2020-10-17 01:18:44 collections:174] 0 files were filtered totalling 0.00 hours
[NeMo I 2020-10-17 01:18:44 collections:173] Dataset loaded with 868 files totalling 1.00 hours
[NeMo I 2020-10-17 01:18:44 collections:174] 0 files were filtered totalling 0.00 hours
[NeMo I 2020-10-17 01:18:44 modelPT:572] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.8, 0.5]
        eps: 1e-08
        grad_averaging: False
        lr: 0.0001
        weight_decay: 0.001
    )
[NeMo I 2020-10-17 01:18:44 lr_scheduler:545] Scheduler "<nemo.core.optim.lr_scheduler.CosineAnnealing object at 0x7fed9cbbe490>" 
    will be used during training (effective maximum steps = 650417) - 
    Parameters : 
    (warmup_steps: null
    warmup_ratio: null
    min_lr: 0.0
    last_epoch: -1
    max_steps: 650417
    )


(Novograd (
 Parameter Group 0
     amsgrad: False
     betas: [0.8, 0.5]
     eps: 1e-08
     grad_averaging: False
     initial_lr: 0.0001
     lr: 0.0001
     weight_decay: 0.001
 ),
 {'scheduler': <nemo.core.optim.lr_scheduler.CosineAnnealing at 0x7fed9cbbe490>,
  'interval': 'step',
  'frequency': 1,
  'monitor': 'val_loss',
  'reduce_on_plateau': False})

In [23]:
from reshuffle_samples import reshuffle_samples
reshuffle_samples(C)

In [ ]:
trainer.fit(model)

[NeMo I 2020-10-17 01:19:24 modelPT:572] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.8, 0.5]
        eps: 1e-08
        grad_averaging: False
        lr: 0.0001
        weight_decay: 0.001
    )
[NeMo I 2020-10-17 01:19:24 lr_scheduler:545] Scheduler "<nemo.core.optim.lr_scheduler.CosineAnnealing object at 0x7fed9cb07290>" 
    will be used during training (effective maximum steps = 650417) - 
    Parameters : 
    (warmup_steps: null
    warmup_ratio: null
    min_lr: 0.0
    last_epoch: -1
    max_steps: 650417
    )


[NeMo W 2020-10-17 01:19:25 nemo_logging:349] /home/catskills/anaconda3/envs/nemo/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning:
    
    Could not log computational graph since the `model.example_input_array` attribute is not set or `input_array` was not given
    
    

  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConvASREncoder                    | 18 M  
2 | spec_augmentation | SpectrogramAugmentation           | 0     
3 | _wer              | WER                               | 0     
4 | decoder           | ConvASRDecoder                    | 50 K  
5 | loss              | CTCLoss                           | 0     


[NeMo I 2020-10-17 01:19:25 wer:148] 
    
[NeMo I 2020-10-17 01:19:25 wer:149] reference:هو هو
[NeMo I 2020-10-17 01:19:25 wer:150] decoded  :ڼثپعپرپرعرږپ


[NeMo W 2020-10-17 01:19:26 nemo_logging:349] /home/catskills/anaconda3/envs/nemo/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:123: UserWarning:
    
    Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
    
    


[NeMo I 2020-10-17 01:19:40 wer:148] 
    
[NeMo I 2020-10-17 01:19:40 wer:149] reference:ځه نو د دې ویلو څه ضرورت وو ما تا ته پرون خبره کړې وه که نه چې داسې کار به کېږي نو بیا دوباره انکار کولو ضرورت څه وو ته ولې داسې کوې په دې کارونو خو ته مخ ته نه ځې
[NeMo I 2020-10-17 01:19:40 wer:150] decoded  :ڼثثثپپآیعخرۍيۍعآعرڅر رپپآپتپپآټپۍپپشئپګپخټتصپپګڼعپپپپخپپټپعًًپټپآئڅږآپکپئپعڼ ًتپپڼ
[NeMo I 2020-10-17 01:19:55 wer:148] 
    
[NeMo I 2020-10-17 01:19:55 wer:149] reference:څه شی
[NeMo I 2020-10-17 01:19:55 wer:150] decoded  :څعع
[NeMo I 2020-10-17 01:20:11 wer:148] 
    
[NeMo I 2020-10-17 01:20:11 wer:149] reference:نه نه هغه مونږ بیا هغسې نه اخلو او
[NeMo I 2020-10-17 01:20:11 wer:150] decoded  :  ع
[NeMo I 2020-10-17 01:20:26 wer:148] 
    
[NeMo I 2020-10-17 01:20:26 wer:149] reference:ښه ښه صحیح ده صحیح ده بس زه پوهه شوم دې ته زمونږ کلي کښې څه بل څه نوم وایي
[NeMo I 2020-10-17 01:20:26 wer:150] decoded  :            ع
[NeMo I 2020-10-17 01:20:42 wer:148] 
    
[NeMo I 2020-10-17 01:20

...

[NeMo I 2020-10-17 01:23:06 wer:148] 
    
[NeMo I 2020-10-17 01:23:06 wer:149] reference:ښه یې تنګ وه
[NeMo I 2020-10-17 01:23:06 wer:150] decoded  :ه ه ه ع
[NeMo I 2020-10-17 01:23:21 wer:148] 
    
[NeMo I 2020-10-17 01:23:21 wer:149] reference:السلام علیکم
[NeMo I 2020-10-17 01:23:21 wer:150] decoded  : ه ع
[NeMo I 2020-10-17 01:23:37 wer:148] 
    
[NeMo I 2020-10-17 01:23:37 wer:149] reference:پچانوې
[NeMo I 2020-10-17 01:23:37 wer:150] decoded  :ه هع
[NeMo I 2020-10-17 01:23:52 wer:148] 
    
[NeMo I 2020-10-17 01:23:52 wer:149] reference:ورکه ورته که در سره وي ورکه
[NeMo I 2020-10-17 01:23:52 wer:150] decoded  :و ه ههې ه هع
[NeMo I 2020-10-17 01:24:06 wer:148] 
    
[NeMo I 2020-10-17 01:24:06 wer:149] reference:خپل پټي کښې به کار وکړي ستا څه شی سره شوق وي که نه چې هر شي سره دې شوق وي که زراعت سره دې شوق وي ماموریت سره دې شوق وي
[NeMo I 2020-10-17 01:24:06 wer:150] decoded  :ه ه ه  ه ه    ه  ه ې ه    ه ه ه ه ه ه  ه ې ه ه ه  ه ه ع
[NeMo I 2020-10-17 01:24:22 wer:148] 
    
[NeMo

[NeMo I 2020-10-17 01:26:08 wer:148] 
    
[NeMo I 2020-10-17 01:26:08 wer:149] reference:هو هو
[NeMo I 2020-10-17 01:26:08 wer:150] decoded  :ع
[NeMo I 2020-10-17 01:26:08 wer:148] 
    
[NeMo I 2020-10-17 01:26:08 wer:149] reference:کرېکټ باره کرېکټ کرېکټ مېچ نه ګورې
[NeMo I 2020-10-17 01:26:08 wer:150] decoded  : کې کې که ه ې ه ه ه وېع
[NeMo I 2020-10-17 01:26:08 wer:148] 
    
[NeMo I 2020-10-17 01:26:08 wer:149] reference:ځای له ور شو که نه د خیره ځه صحیح مونږ به دلته لږ چکر کړو او خوښه دې نه ده چې تاسو دواړه را شئ دلته حبیب صاحب دې ځای ته
[NeMo I 2020-10-17 01:26:08 wer:150] decoded  :ه ه  ه ه ه   ه مه ه کې ک  ه ه ه ې ه  اه ا ې ه ې ېې  هع
[NeMo I 2020-10-17 01:26:08 wer:148] 
    
[NeMo I 2020-10-17 01:26:08 wer:149] reference:خوشحاله به شو ډېر
[NeMo I 2020-10-17 01:26:08 wer:150] decoded  :ه ه ده  ېع
[NeMo I 2020-10-17 01:26:08 wer:148] 
    
[NeMo I 2020-10-17 01:26:08 wer:149] reference:ښه
[NeMo I 2020-10-17 01:26:08 wer:150] decoded  :ه ع
[NeMo I 2020-10-17 01:26:08 wer:148] 

[NeMo I 2020-10-17 01:26:11 wer:148] 
    
[NeMo I 2020-10-17 01:26:11 wer:149] reference:ګوره اقتصاد په د کټ په پښو غځولو باندې نه ښه کېږي
[NeMo I 2020-10-17 01:26:11 wer:150] decoded  :ه ه ې ه ه که کې ه کو هه وه ه ه کېه کېع
[NeMo I 2020-10-17 01:26:11 wer:148] 
    
[NeMo I 2020-10-17 01:26:11 wer:149] reference:خامخا اوس به ځې
[NeMo I 2020-10-17 01:26:11 wer:150] decoded  :ه ه و  يع
[NeMo I 2020-10-17 01:26:11 wer:148] 
    
[NeMo I 2020-10-17 01:26:11 wer:149] reference:ښه دې ما او عمران نن ولاړو اکاډمي ته
[NeMo I 2020-10-17 01:26:11 wer:150] decoded  :ه ه هه ه ه ه ې کې ې ه هع
[NeMo I 2020-10-17 01:26:11 wer:148] 
    
[NeMo I 2020-10-17 01:26:11 wer:149] reference:اوس دې وخت کښې خو باران هم زیات وي خدای غرق کړی
[NeMo I 2020-10-17 01:26:11 wer:150] decoded  : ه  انه  ې و ا  ه ک ع
[NeMo I 2020-10-17 01:26:11 wer:148] 
    
[NeMo I 2020-10-17 01:26:11 wer:149] reference:ښه الحمد الله پلار دې ښه دی وروڼه دې ټول ښه دي
[NeMo I 2020-10-17 01:26:11 wer:150] decoded  :ه ه هه ا ه ه ده ې ه ي


Epoch 00001: saving model to /home/catskills/Desktop/openasr20/save/nemo_pashto/amharic_20201017_011238_108037_epoch=01.ckpt


[NeMo I 2020-10-17 01:26:29 wer:148] 
    
[NeMo I 2020-10-17 01:26:29 wer:149] reference:هلو
[NeMo I 2020-10-17 01:26:29 wer:150] decoded  :هع
[NeMo I 2020-10-17 01:26:44 wer:148] 
    
[NeMo I 2020-10-17 01:26:44 wer:149] reference:ښوولی شئ ښوولی شئ
[NeMo I 2020-10-17 01:26:44 wer:150] decoded  :هه ه ه ع


## DEV translation

In [ ]:
from Cfg import Cfg
from glob import glob
from package_DEV import package_DEV
from load_pretrained_amharic_model import load_pretrained_amharic_model
version='113'
C = Cfg('NIST', 16000, 'pashto', 'dev', version)
model = load_pretrained_amharic_model(C, 0)
files=list(sorted(glob(f'{C.audio_split_dir}/*.wav')))
translations=model.transcribe(paths2audio_files=files, batch_size=32)